In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.09158577024936676
epoch:  1, loss: 0.06394219398498535
epoch:  2, loss: 0.05308891460299492
epoch:  3, loss: 0.045529838651418686
epoch:  4, loss: 0.03201550245285034
epoch:  5, loss: 0.028594907373189926
epoch:  6, loss: 0.027501296252012253
epoch:  7, loss: 0.026138441637158394
epoch:  8, loss: 0.02559693716466427
epoch:  9, loss: 0.0254640132188797
epoch:  10, loss: 0.025071555748581886
epoch:  11, loss: 0.0240323506295681
epoch:  12, loss: 0.023709913715720177
epoch:  13, loss: 0.023540427908301353
epoch:  14, loss: 0.023364083841443062
epoch:  15, loss: 0.02319788560271263
epoch:  16, loss: 0.022997938096523285
epoch:  17, loss: 0.022748541086912155
epoch:  18, loss: 0.022612858563661575
epoch:  19, loss: 0.022460652515292168
epoch:  20, loss: 0.022332023829221725
epoch:  21, loss: 0.022201498970389366
epoch:  22, loss: 0.022159777581691742
epoch:  23, loss: 0.022060180082917213
epoch:  24, loss: 0.022010117769241333
epoch:  25, loss: 0.021956801414489746
epoch:

In [6]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.18761606514453888
epoch:  1, loss: 0.10066165030002594
epoch:  2, loss: 0.06383821368217468
epoch:  3, loss: 0.03253023326396942
epoch:  4, loss: 0.028588728979229927
epoch:  5, loss: 0.027696512639522552
epoch:  6, loss: 0.027269072830677032
epoch:  7, loss: 0.02692682109773159
epoch:  8, loss: 0.026460733264684677
epoch:  9, loss: 0.025815337896347046
epoch:  10, loss: 0.025243690237402916
epoch:  11, loss: 0.024593718349933624
epoch:  12, loss: 0.024219801649451256
epoch:  13, loss: 0.02368350140750408
epoch:  14, loss: 0.02355155162513256
epoch:  15, loss: 0.02268300950527191
epoch:  16, loss: 0.022643180564045906
epoch:  17, loss: 0.022183038294315338
epoch:  18, loss: 0.02216476760804653
epoch:  19, loss: 0.021751604974269867
epoch:  20, loss: 0.021504821255803108
epoch:  21, loss: 0.021483253687620163
epoch:  22, loss: 0.02130015566945076
epoch:  23, loss: 0.021171094849705696
epoch:  24, loss: 0.02108692191541195
epoch:  25, loss: 0.0210048146545887
epoch:  2